In [1]:
import league
import players
import statistics
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots
from prettytable import PrettyTable

PLAYERS = players.Players()
LEAGUE = league.League(13, "649912836461539328", PLAYERS)

# Playoff Probability (part 3)

With the playoffs starting next week, I will do one more notebook on playoff probabilities. Most everyone is still alive and fighting for a better playoff seed!

```
Noe's Loyal Lighting Salesman Division
1. Nahome (18-6)
2. Ben (11-17)
3. Stephane (9-17)
4. Noe (9-17)
5. Will (9-17)

Noah's SideThiccs
1. Praveen (18-8)
2. Stian (17-9)
3. Noah (14-11)
4. Tom (13-13)
5. Tegran (10-14)
```

Power Rankings for this week are as follows

```
1. Stian
2. Nahome
3. Praveen
4. Noah
5. Tom
6. Ben
7. Stephane
8. Will
9. Tegran
10. Noe
```

After his eigth straight win, Stian vaults Nahome into the number 1 power ranking position heading into the last week of the season. Will, Stephane, and Noe are all down near the bottom of the pack but all have the same record, giving them everything to play for this final week.

## Methodology

I'll use the same methodology as before, so if you want to see it check out the first notebook I did on playoff probabilities. This week I'll simply compare the new projections to last weeks and see who is improving their odds. An example simulation is shown below.

In [2]:
LEAGUE.sim_remaining_season(np.random.normal)
results = LEAGUE.simmed_results()

for division in results:
    division = sorted(division, reverse=True)
    for t in division:
        print(LEAGUE.name_pairings[LEAGUE.roster_pairings[t[4]]], t[0], t[1], t[2])
    print()

AssNTitties 22 6 1855.4726527979647
My Goat Loves You 11 17 1730.719737247077
billcap 10 18 1530.653345567191
RB sanctuary 9 19 1598.454041979716
the noés 9 19 1534.0481020074105

Tree Leaves 20 8 1812.4378878240966
tealeaves 19 9 1833.0236507599616
SideThicc #2 15 13 1718.2975165196522
Thicc King 15 13 1689.8414321548967
The Brutherhood 10 18 1635.1244150338684



To simulate 100,000 seasons this time!

In [12]:
simulation = []
for i in range(100000):
    LEAGUE.sim_remaining_season(np.random.normal)
    simulation.append(LEAGUE.simmed_results())

Now lets see where every team finished in this new simulation.

In [13]:
final_standing = {t.roster_id: [0,0,0,0,0] for t in LEAGUE.standings}
probabilities = {t.roster_id: 0 for t in LEAGUE.standings}

for result in simulation:
    for division in result:
        division = sorted(division, reverse=True)
        for i, t in enumerate(division):
            final_standing[t[4]][i] += 1

for roster_id in final_standing:
    print(LEAGUE.name_pairings[LEAGUE.roster_pairings[roster_id]], final_standing[roster_id])
    probabilities[roster_id] = sum(
        final_standing[roster_id][0:3]) / sum(final_standing[roster_id]) * 100
    print(sum(final_standing[roster_id][0:3]) / sum(final_standing[roster_id]))

finish_probabilities = {roster_id: [(num / sum(final_standing[roster_id])) * 100 for num in final_standing[roster_id]] for roster_id in final_standing}

for id in probabilities:
    probabilities[id] = round(probabilities[id], 2)

for id in finish_probabilities:
    finish_probabilities[id] = [round(prob, 2) for prob in finish_probabilities[id]]

AssNTitties [100000, 0, 0, 0, 0]
1.0
Tree Leaves [70336, 29664, 0, 0, 0]
1.0
tealeaves [29664, 70336, 0, 0, 0]
1.0
Thicc King [0, 0, 47605, 52395, 0]
0.47605
SideThicc #2 [0, 0, 52395, 47605, 0]
0.52395
My Goat Loves You [0, 100000, 0, 0, 0]
1.0
The Brutherhood [0, 0, 0, 0, 100000]
0.0
RB sanctuary [0, 0, 34465, 58506, 7029]
0.34465
the noés [0, 0, 32201, 24192, 43607]
0.32201
billcap [0, 0, 33334, 17302, 49364]
0.33334


Now lets visualize these finish positions as bars of outcomes

In [14]:
fig = make_subplots(rows=5, cols=2, shared_yaxes=True,
                    subplot_titles=LEAGUE.get_names_for_plotting())

teams = [team for team in LEAGUE.standings]

for i in range(5):
    for j in range(1, 3):
        current = teams[(i+i+j)-1]
        fig.add_trace(
            go.Bar(y=final_standing[current.roster_id],
                   x=["1st", "2nd", "3rd", "4th", "5th"]),
            i+1, j
        )

fig.update_layout(title="Likelihood of each Teams' Final Standing",
                  showlegend=False,
                  height=2000, width=1500
                  )

fig.show()

Below I print out the calculated probability that each team makes the playoffs and the probability they finish in each of the five division positions. I also include the change in playoff odds from last week. 

In [15]:
t = PrettyTable(['Team Name', "Percent Chance of Making Playoffs", "Percent Change from Last Week"])
last_week = {1: 93.29, 2: 0.59, 3: 97.99, 4: 99.63, 5: 36.41,
             6: 56.82, 7: 100, 8: 44.96, 9: 7.66, 10: 62.63}
for team in LEAGUE.standings:
    t.add_row([team.name, probabilities[team.roster_id], round(probabilities[team.roster_id] - last_week[team.roster_id], 2)])
    
print(t)

+-------------------+-----------------------------------+-------------------------------+
|     Team Name     | Percent Chance of Making Playoffs | Percent Change from Last Week |
+-------------------+-----------------------------------+-------------------------------+
|    AssNTitties    |               100.0               |              0.0              |
|    Tree Leaves    |               100.0               |              0.37             |
|     tealeaves     |               100.0               |              2.01             |
|     Thicc King    |                47.6               |              2.64             |
|    SideThicc #2   |                52.4               |             -4.42             |
| My Goat Loves You |               100.0               |              6.71             |
|  The Brutherhood  |                0.0                |             -0.59             |
|    RB sanctuary   |               34.47               |             -28.16            |
|      the

In [16]:
t = PrettyTable(['Team Name', "P(Finishing 1st in Division)",
                "P(Finishing 2nd in Division)", "P(Finishing 3rd in Division)", "P(Finishing 4th in Division)", "P(Finishing 5th in Division)"])
for team in LEAGUE.standings:
    t.add_row([team.name, finish_probabilities[team.roster_id]
              [0], finish_probabilities[team.roster_id][1], finish_probabilities[team.roster_id][2], finish_probabilities[team.roster_id][3], finish_probabilities[team.roster_id][4]])

print(t)


+-------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+
|     Team Name     | P(Finishing 1st in Division) | P(Finishing 2nd in Division) | P(Finishing 3rd in Division) | P(Finishing 4th in Division) | P(Finishing 5th in Division) |
+-------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+
|    AssNTitties    |            100.0             |             0.0              |             0.0              |             0.0              |             0.0              |
|    Tree Leaves    |            70.34             |            29.66             |             0.0              |             0.0              |             0.0              |
|     tealeaves     |            29.66             |            70.34             |             0.0              | 